<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Réseaux-neuronaux-récurrents" data-toc-modified-id="Réseaux-neuronaux-récurrents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Réseaux neuronaux récurrents</a></span></li></ul></div>

# Réseaux neuronaux récurrents


On a déjà vu les les réseaux denses (réseaux de neurones classiques) et les réseaux convolutionnels. 
=> Ils ont en commun de n'avoir *aucune mémoire*, chaque échantillon est traité de façon indépendante

Ce sont des réseaux de neurones dit « à propagation avant » (*feedforward*)
Avec ces réseaux

On ne peut pas vraiment utiliser des séquences ou des séries temporelles, il manque quelque chose.

## Traitement séquentiel

Quand vous lisez un texte, une phrase, on lit mot par mot tout en ayant en mémoire ce qui précède, ce qui permet d'avoir le *sens* de ce qui est lu.

Les processus biologiques intelligents traitent l'information incrémentalement tout en conservant un modèle interne de ce qui est traité, construit à partir des informations passées et mis à jour continuellement au fur et à mesure que de nouvelles informations arrivent.

## RNN

C'est justement l'objet des *RNN*, dans une forme extrêmement simplifiée: on traite des séquences et on maintient un *état* qui contient l'information de ce qui est mis à jour. Un *RNN* a une boucle interne, ce n'est plus du *feedforward*.

Lorsqu'on a terminé la séquence, on remet l'état à zéro. 

Les séquences deviennent les *échantillons* (ou individus) de notre jeu de données.

La différence avec les *feedforward* est que l'échantillon n'est pas traité en une seule passe :
le réseau *boucle* sur les éléments de la séquence.

<img src=../_static/img/RNN/RNN.svg width=400 />

## Implémentation naïve en numpy

On encode un tenseur 2D (matrice) de dimension `(timesteps,input_features)`. 

- On boucle sur les pas de temps (*timesteps*)

- À chaque `timestep` on considére l'état en `t` et l'entrée en `t` (qui de longueur de la séquence, ici `(input_features,)`)

- On combine l'entrée et l'état pour obtenir la nouvelle sortie à `t` qui devient le nouvel état pour la prochaine itération.

On initialise l'état à zéro pour le premier pas de temps.

In [ ]:
state_t = 0 # l'état à t
for input_t in input_sequence: # itération sur des séquences d'éléments
    output_t = f(input_t, state_t)
    state_t = output_t # la sortie précédente devient l'état pour l'itération suivante

Si on veut, on peut exprimer sous forme affine la fonction $f$ et que l'on considère les paramètres $W,U,B$

$$f(I_t,S_t) = \sigma{}(W\cdot{}I_t + U\cdot{}S_t + B)$$

![](../_static/img/RNN/RNNunrolled.jpg)

In [ ]:
state_t = 0
for input_t in input_sequence:
    output_t = activation(dot(W, input_t) + dot(U, state_t) + b)
    state_t = output_t

In [1]:
import numpy as np

timesteps = 100 # nombre d'intervalles temporels
input_features = 32 # dimension de l'espace d'entrée des caractéristiques
output_features = 64 # dimension de l'espace de sortie des caractéristiques

In [2]:
inputs = np.random.random((timesteps, input_features)) # entrées aléatoires pour l'example

state_t = np.zeros((output_features,)) # on initialise l'état, tout à zéro

In [3]:
W = np.random.random((output_features, input_features))  #
U = np.random.random((output_features, output_features)) # création aléatoire de matrices de poids
b = np.random.random((output_features,))                 #

In [4]:
successive_outputs = []
for input_t in inputs: # vecteurs de forme (input_features,)
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b) # combine l'entrée 
                                                                    # avec l'état actuel (la sortie précédente)
                                                                    # pour obtenir la nouvelle sortie

    successive_outputs.append(output_t) # stocke cette sortie dans une liste

    state_t = output_t # mets à jours l'état du réseau pour le prochain intervalle temporel

final_output_sequence = np.concatenate(successive_outputs, axis=0) # le résultat final est un
                                                                   # vecteur 2D de forme (timesteps,output_features)

## LTSM (Long Term Short Memory)

Ajoutons au réseau récurrent une « porteuse » (*carry*) qui va se rajouter à l'état sur le neurone.

![unroll](../_static/img/RNN/LTSM.jpg)

Attention, la subtilité est que la prochaine valeur de la porteuse est calculée à chaque itération, et que pour ce faire on effectue, non plus une mais 3 transformations. Toutes les trois on la forme :

In [ ]:
y = activation(dot(state_t, U) + dot(input_t, W) + b)

Mais les trois transformations ont leur propres matrices de paramètres! On va les indexer par `i`, `f`, `k`.

In [ ]:
output_t = activation(dot(state_t, Uo) + dot(input_t, Wo) + dot(C_t, Vo) + bo)

i_t = activation(dot(state_t, Ui) + dot(input_t, Wi) + bi)
f_t = activation(dot(state_t, Uf) + dot(input_t, Wf) + bf)
k_t = activation(dot(state_t, Uk) + dot(input_t, Wk) + bk)

On obtient la prochaine porteuse en combinant les 3 :

In [ ]:
c_t+1 = i_t * k_t + c_t * f_t

### « Interprétation » du LTSM

On peut dire que par la multiplication de `c_t` et `f_t`, on effectue une sorte de filtre sur le flot de données de la porteuse. 

Dans le même temps, `i_t` et `k_t` apportent des informations sur le présent, et mettent à jour cette porteuse avec celles-ci.

## Le RNN de PyTorch

`torch.nn.RNN(*args, **kwargs)`

Applies a multi-layer Elman RNN with $tanh$ or $ReLU$ non-linearity to an input sequence.

For each element in the input sequence, each layer computes the following function:

$$h_t = \text{tanh}(w_{ih} x_t + b_{ih} + w_{hh} h_{(t-1)} + b_{hh})$$

where $h_t$ is the hidden state at time `t`, $x_t$ is
the input at time `t`, and $h_{(t-1)}$ is the hidden state of the
previous layer at time `t-1` or the initial hidden state at time `0`.

If `nonlinearity` is `'relu'`, then `ReLU` is used instead of `tanh`.

### Args:
- input_size: The number of expected features in the input `x`
- hidden_size: The number of features in the hidden state `h`
- num_layers: Number of recurrent layers. E.g., setting ``num_layers=2`` would mean stacking two RNNs together to form a `stacked RNN`, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
- nonlinearity: The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
- bias: If ``False``, then the layer does not use bias weights `b_ih` and `b_hh`. Default: ``True``
- batch_first: If ``True``, then the input and output tensors are provided as `(batch, seq, feature)`. Default: ``False``
- dropout: If non-zero, introduces a `Dropout` layer on the outputs of each RNN layer except the last layer, with dropout probability equal to `dropout`. Default: 0
- bidirectional: If ``True``, becomes a bidirectional RNN. Default: ``False``

### Inputs: 
    input, h_0
- **input** of shape `(seq_len, batch, input_size)`: tensor containing the features of the input sequence. The input can also be a packed variable length sequence. See `torch.nn.utils.rnn.pack_padded_sequence` or `torch.nn.utils.rnn.pack_sequence` for details.
- **h_0** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided. If the RNN is bidirectional, num_directions should be 2, else it should be 1.

### Outputs: 
    output, h_n
- **output** of shape `(seq_len, batch, num_directions * hidden_size)`: tensor containing the output features (`h_k`) from the last layer of the RNN, for each `k`.  If a `torch.nn.utils.rnn.PackedSequence` has been given as the input, the output will also be a packed sequence.
For the unpacked case, the directions can be separated using ``output.view(seq_len, batch, num_directions, hidden_size)``, with forward and backward being direction `0` and `1` respectively. Similarly, the directions can be separated in the packed case.
- **h_n** (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for `k = seq_len`.

Like *output*, the layers can be separated using ``h_n.view(num_layers, num_directions, batch, hidden_size)``.

### Attributes:
- weight_ih_l[k]: the learnable input-hidden weights of the k-th layer, of shape `(hidden_size * input_size)` for `k = 0`. Otherwise, the shape is `(hidden_size * hidden_size)`
- weight_hh_l[k]: the learnable hidden-hidden weights of the k-th layer, of shape `(hidden_size * hidden_size)`
- bias_ih_l[k]: the learnable input-hidden bias of the k-th layer, of shape `(hidden_size)`
- bias_hh_l[k]: the learnable hidden-hidden bias of the k-th layer, of shape `(hidden_size)`

All the weights and biases are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{\text{hidden\_size}}$

In [12]:
import torch
from torch import nn

rnn = nn.RNN(5, 3, 1)
input = torch.randn(1, 1, 5)
input

tensor([[[-1.0569,  0.3272,  1.1545, -0.5483, -0.8013]]])

In [13]:
h0 = torch.randn(1, 1, 3)
h0

tensor([[[-0.1285,  0.2534, -0.1705]]])

In [20]:
output, h1 = rnn(input, h0)
print(output)
print(h1)

tensor([[[-0.3679,  0.7967,  0.1165]]], grad_fn=<StackBackward>)
tensor([[[-0.3679,  0.7967,  0.1165]]], grad_fn=<StackBackward>)
